# Multi-Label Classification

## 1. Imports

In [1]:
import ast
import os
import pickle
import random

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from numpy.typing import ArrayLike
from sklearn import preprocessing
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from tqdm.contrib.itertools import product
from tqdm.notebook import tqdm

load_dotenv()

RANDOM_STATE = int(os.getenv("RANDOM_STATE"))
TEST_SIZE = float(os.getenv("TEST_SIZE"))

os.chdir(os.getenv("ROOT"))
np.random.seed(seed=RANDOM_STATE)
random.seed(a=RANDOM_STATE)

import MLC
from notebooks.utils import (
    assess,
    extract_models,
    preprocess_texts,
    replace_text_components,
    translate_source_categories,
)

In [2]:
class ModuleSystem(BaseEstimator, ClassifierMixin):
    def __init__(self, models, mlb):
        self.models = models
        self.mlb = mlb

    def predict(self, X_test: ArrayLike) -> ArrayLike:
        """
        Predict labels for each component using the chosen model in the dictionary.

        Parameters
        ----------
        X_test : ArrayLike
            Test data.

        Returns
        -------
        Y_pred : ArrayLike
            Predicted labels for each component.
        """
        n_samples = X_test.shape[0]
        n_components = len(self.models)
        Y_pred = np.zeros((n_samples, n_components))

        for idx, model_info in self.models.items():
            model = model_info["model"]
            Y_pred[:, int(idx)] = (model.predict(X_test))[:, int(idx)]

        return Y_pred

## 2. Configurations & Constants

In [3]:
GENERAL_MAPPINGS = {
    "computer security": "computer security",
    "operating systems": "operating systems",
    "software": "software",
    "programming languages": "software",
    "hardware": "hardware",
    "electronic components": "hardware",
    "networking": "networking",
    "internet technology": "networking",
}

## Model Training

In [4]:
CLASSIFICATION_METHODS = {
    "br": MLC.BRClassifier,
    "clr": MLC.CLRClassifier,
    "cc": MLC.CCClassifier,
    "lp": MLC.LPClassifier,
    "pst": MLC.PStClassifier,
    #'cdn': MLC.CDNClassifier,
    "mbr": MLC.MBRClassifier,
    "rakel": MLC.RAkELClassifier,
    "homer": MLC.HOMERClassifier,
}

CLASSIFICATION_ALGORITHM = {
    "lr": LogisticRegression(
        solver="liblinear", max_iter=10000, random_state=RANDOM_STATE
    ),
    "gnb": GaussianNB(),
    #"dt": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "rf": RandomForestClassifier(random_state=RANDOM_STATE),
}

EMBEDDING_SOURCES = [
    {
        "name": "GloVe.6B.50D",
        "model-path": os.getenv("GLOVE_6B_50D_PATH"),
        "embedding-dim": 50,
    },
    {
        "name": "GloVe.6B.100D",
        "model-path": os.getenv("GLOVE_6B_100D_PATH"),
        "embedding-dim": 100,
    },
    {
        "name": "GloVe.6B.200D",
        "model-path": os.getenv("GLOVE_6B_200D_PATH"),
        "embedding-dim": 200,
    },
    {
        "name": "GloVe.6B.300D",
        "model-path": os.getenv("GLOVE_6B_300D_PATH"),
        "embedding-dim": 300,
    },
    {
        "name": "DistilRoBERTa",
        "model-path": os.getenv("DISTILROBERTA_PATH"),
        "embedding-dim": None,
    },
    {"name": "SBERT", "model-path": os.getenv("SBERT_PATH"), "embedding-dim": None},
    {
        "name": "ATT&CK-BERT",
        "model-path": os.getenv("ATTACK_BERT_PATH"),
        "embedding-dim": None,
    },
]

print()
SYSTEM_MODULES = [
    {
        "level": "general",
        "targets": {
            "computer security": "computer security",
            "operating systems": "operating systems",
            "software": "software",
            "programming languages": "software",
            "hardware": "hardware",
            "electronic components": "hardware",
            "networking": "networking",
            "internet technology": "networking",
        }
    },
    #{
    #    "level": "computer security",
    #    "targets": {
    #        "network security": "network security",
    #        "antivirus and malware": "antivirus and malware",
    #    },
    #},
    #{
    #    "level": "operating systems",
    #    "targets": {
    #        "mac os": "mac os",
    #        "windows": "windows",
    #        "unix": "unix",
    #        "linux": "linux",
    #    },
    #},
    #{"level": "software", "targets": {"databases": "databases"}},
    #{
    #    "level": "hardware",
    #    "targets": {
    #        "computer": "computer",
    #        "computer components": "computer components",
    #        "computer networking": "computer networking",
    #    },
    #},
]

In [5]:
with open(os.getenv("TEMP_CATEGORIES_TRAIN_CSV"), "rb") as f:
    train = pd.read_csv(f)
    train = train.assign(
        tweet=lambda df: df["tweet"].apply(func=ast.literal_eval),
        watson=lambda df: df["watson"].apply(func=ast.literal_eval),
        urls=lambda df: df["urls"].apply(func=ast.literal_eval),
        watson_list=lambda df: df["watson_list"].apply(func=ast.literal_eval),
        target=lambda df: df["target"].apply(func=ast.literal_eval),
    )
    train["text"] = [replace_text_components(t) for t in train["text"]]
    for idx, t in enumerate(train['relevant']):
        if not t and 'other' not in train.loc[idx, "target"]:
            train.loc[idx, "target"] = ["other"]

best_module = None
for module_config in tqdm(SYSTEM_MODULES, desc="Modules", unit="module"):
    accuracy = 0
    best_module = None

    for embedder in tqdm(EMBEDDING_SOURCES, desc="Embedders", unit="embedder"):
        tweets = train.assign(
            target=lambda df: df["target"].apply(
                func=translate_source_categories, args=(module_config["targets"],)
            )
        )

        if module_config["level"] != "general":
            tweets = tweets[
                tweets["watson_list"].apply(lambda x: module_config["level"] in x)
            ].reset_index(drop=True)

        tweets["text"] = [replace_text_components(t) for t in tweets["text"]]
        texts = tweets["text"]
        x = preprocess_texts(
            list_str=texts,
            model_path=embedder["model-path"],
            embedding_dim=embedder["embedding-dim"],
        )

        targets = tweets["target"]
        mlb = preprocessing.MultiLabelBinarizer()
        y = mlb.fit_transform(y=targets)
        _, y_mcp = np.unique(ar=y, axis=0, return_inverse=True)

        X_train, X_val, y_train, y_val = train_test_split(
            x,
            y,
            test_size=TEST_SIZE,
            random_state=RANDOM_STATE,
            shuffle=True,
            stratify=y_mcp,
        )

        models = {}
        trained_models = []
        for method_name, algorithm_name in product(
                CLASSIFICATION_METHODS,
                CLASSIFICATION_ALGORITHM,
                desc="Methods & Algorithms",
                unit="pair",
        ):
            model = CLASSIFICATION_METHODS[method_name](
                CLASSIFICATION_ALGORITHM[algorithm_name]
            )
            try:
                model.fit(X_train, y_train)
                model_name = f"{method_name.upper()}[{algorithm_name.upper()}] - {embedder['name']}"
            except Exception as e:
                print(f"{model} failed to fit due to {e}")
                model_name = None
                continue
            if model_name is not None:
                trained_models.append(model_name)
                models[model_name] = {
                    "model": model,
                    "assess": model.evaluate(X_val, y_val),
                }

        top_performing_models = extract_models(mlb.classes_, models, "f1-score")
        module = ModuleSystem(top_performing_models, mlb)
        y_val_pred = module.predict(X_val)
        validation_results = assess(y_val, y_val_pred)

        # Convert the predicted labels back to the original format using MultiLabelBinarizer
        if validation_results["accuracy"] > accuracy:
            accuracy = validation_results["accuracy"]
            best_module = [m["name"] for m in top_performing_models.values()]
            print("==========================================================")
            print(f"{module_config["level"].capitalize()} Performance")
            print("==========================================================")
            print(f"Accuracy:\t{validation_results['accuracy'] * 100:.2f}%")
            print(f"Hamming Loss:\t{validation_results['hamming_loss']:.3f}")
            report = pd.DataFrame(validation_results["report"])
            report.columns = list(module.mlb.classes_) + [
                "micro avg",
                "macro avg",
                "weighted avg",
                "samples avg",
            ]
            report = (
                report.transpose()
                .map(lambda x: f"{x:.2f}" if isinstance(x, float) else x)
                .to_string()
            )
            print(f"{report}")
            print("==========================================================\n")

        model_dictionary_filename = f"{os.getenv("MAPPINGS_DIR")}/{module_config["level"].capitalize()}.{embedder['name']}"
        with open(f"{model_dictionary_filename}.pickle", "wb") as f:
            pickle.dump(models, f)

        del tweets, texts, x, targets, mlb, y, y_mcp, X_train, X_val, y_train, y_val, models, trained_models, top_performing_models, module, y_val_pred, validation_results, model_dictionary_filename

    module_filename = f"{os.getenv("MODELS_DIR")}/{''.join(module_config["level"].capitalize().split(' '))}"
    with open(f"{module_filename}.pickle", "wb") as f:
        pickle.dump(best_module, f)

Modules:   0%|          | 0/1 [00:00<?, ?module/s]

Embedders:   0%|          | 0/7 [00:00<?, ?embedder/s]

Methods & Algorithms:   0%|          | 0/24 [00:00<?, ?pair/s]

HOMERClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                  random_state=42,
                                                  solver='liblinear')) failed to fit due to This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)
General Performance
Accuracy:	76.30%
Hamming Loss:	0.080
                  precision recall f1-score  support   auc
computer security      0.82   0.59     0.69   863.00  0.77
hardware               0.85   0.49     0.62   271.00  0.74
networking             0.86   0.44     0.58   129.00  0.72
operating systems      0.96   0.64     0.77   108.00  0.82
other                  0.80   0.94     0.87  1966.00  0.79
software               0.86   0.68     0.76   581.00  0.83
micro avg              0.82   0.77     0.79  3918.00  0.86
macro avg              0.86   0.63     0.71  3918.00  0.78
weighted avg           0.82   0.77     0.78  3918.00  0.79
samples avg      

Methods & Algorithms:   0%|          | 0/24 [00:00<?, ?pair/s]

...

HOMERClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                  random_state=42,
                                                  solver='liblinear')) failed to fit due to This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)


Methods & Algorithms:   0%|          | 0/24 [00:00<?, ?pair/s]

HOMERClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                  random_state=42,
                                                  solver='liblinear')) failed to fit due to This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)


Methods & Algorithms:   0%|          | 0/24 [00:00<?, ?pair/s]

HOMERClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                  random_state=42,
                                                  solver='liblinear')) failed to fit due to This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)


In [63]:
with open(f"{os.getenv("MAPPINGS_DIR")}/General.ATT&CK-BERT.pickle", "rb") as f:
    general_models = pickle.load(f)

tweets = train.assign(
    target=lambda df: df["target"].apply(
        func=translate_source_categories, args=(module_config["targets"],)
    )
)

if SYSTEM_MODULES[0]["level"] != "general":
    tweets = tweets[
        tweets["watson_list"].apply(lambda x: module_config["level"] in x)
    ].reset_index(drop=True)

targets = tweets["target"]
mlb = preprocessing.MultiLabelBinarizer()
mlb.fit(y=targets)
general_models = extract_models(mlb.classes_, general_models, "f1-score")
module = ModuleSystem(general_models, mlb)
del tweets, targets, mlb, general_models
module

ModuleSystem(mlb=MultiLabelBinarizer(),
             models={'0': {'model': MBRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'MBR[RF] - ATT&CK-BERT',
                           'performance': 0.7082785808147175},
                     '1': {'model': LPClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'LP[RF] - ATT&CK-BERT',
                           'performance': 0.6403712296983759},
                     '2': {'model':...
                           'performance': 0.797979797979798},
                     '4': {'model': BRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'BR[RF] - ATT&CK-BERT',
                           'performance': 0.8753977968176254},
                     '5': {'model': RAkELClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                      random_state=42,
                                                                                      solver='liblinear'),
                                                    n_estimators=12),
                           'name': 'RAKEL[LR] - ATT&CK-BERT',
                           'performance': 0.7576853526220615}})

In [64]:
module_filename = f"{os.getenv("MODELS_DIR")}/{''.join(SYSTEM_MODULES[0]["level"].capitalize().split(' '))}"
with open(f"{module_filename}.pickle", "wb") as f:
    pickle.dump(module, f)
del module_filename, module

# 7. Model Evaluation

In [14]:
with open(os.getenv("TEMP_CATEGORIES_TEST_CSV"), "rb") as f:
    test_tweets = pd.read_csv(f)
    test_tweets = test_tweets.assign(
        tweet=lambda df: df["tweet"].apply(func=ast.literal_eval),
        watson=lambda df: df["watson"].apply(func=ast.literal_eval),
        urls=lambda df: df["urls"].apply(func=ast.literal_eval),
        watson_list=lambda df: df["watson_list"].apply(func=ast.literal_eval),
        target=lambda df: df["target"].apply(func=ast.literal_eval),
    )
    test_tweets['text'] = [replace_text_components(t) for t in test_tweets['text']]

print(f"Threat Tweets: {len(test_tweets)}")
test_tweets.head()

Threat Tweets: 4067


,Unnamed: 0,_id,date,id,relevant,text,tweet,type,watson,annotation,urls,destination_url,valid_certificate,watson_list,target
0,1533,b'5b8e2e39bb325e2996c66aab',2018-09-04 07:03:16+00:00,1036872307057352704,True,threatmeter: irisgraphic sql injection vulnera...,{'created_at': 'Tue Sep 04 07:03:16 +0000 2018...,vulnerability,{'technology and computing': {'computer securi...,threat,[https://ift.tt/2wNsX91],https://cxsecurity.com/issue/WLB-2018090028,True,"[technology and computing, computer security, ...","[antivirus and malware, software, databases, c..."
1,59,b'5b887f63bb325e65fa7e7922',2018-08-30 23:36:02+00:00,1035310204102078464,False,Download this how-to guide on preparing for an...,{'created_at': 'Thu Aug 30 23:36:02 +0000 2018...,ransomware,{'technology and computing': {'computer securi...,business,[http://bit.ly/2wuUTyT],https://logrhythm.com/the-ransomware-threat-eb...,True,"[technology and computing, computer security, ...","[antivirus and malware, computer security]"
2,10824,b'5b92cad0bb325e521c5ae40a',2018-09-07 19:00:31+00:00,1038139971562078208,True,The US Justice Department is poised to charge ...,{'created_at': 'Fri Sep 07 19:00:31 +0000 2018...,ransomware,"{'law, govt and politics': {'government': {}, ...",threat,[https://twitter.com/i/web/status/103813997156...,https://twitter.com/i/web/status/1038139971562...,True,"[law, govt and politics, government, armed for...","[antivirus and malware, computer security]"
3,2104,b'5b90af51bb325e70071868e2',2018-09-06 04:38:41+00:00,1037560694789554176,False,"3) Control and monitor compliance. Also, in...",{'created_at': 'Thu Sep 06 04:38:41 +0000 2018...,general,{'finance': {'personal finance': {'insurance':...,NaN,[https://twitter.com/i/web/status/103756069478...,https://twitter.com/i/web/status/1037560694789...,True,"[finance, personal finance, insurance, busines...",[other]
4,12292,b'5b935e61bb325e521c5aeab9',2018-09-08 05:30:09+00:00,1038298421893967877,True,"Ransomware down, but not out, report reveals",{'created_at': 'Sat Sep 08 05:30:09 +0000 2018...,ransomware,{'business and industrial': {'company': {'annu...,threat,[http://fsecure.smh.re/4AS],http://fsecure.smh.re/4AS,True,"[business and industrial, company, annual repo...","[databases, software]"


In [15]:
texts = test_tweets['text']
targets = test_tweets['target']

In [7]:
with open("models/General.pickle", "rb") as f:
    general_mod = pickle.load(f)

general_mod

ModuleSystem(mlb=MultiLabelBinarizer(),
             models={'0': {'model': MBRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'MBR[RF] - ATT&CK-BERT',
                           'performance': 0.7082785808147175},
                     '1': {'model': LPClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'LP[RF] - ATT&CK-BERT',
                           'performance': 0.6403712296983759},
                     '2': {'model':...
                           'performance': 0.797979797979798},
                     '4': {'model': BRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'BR[RF] - ATT&CK-BERT',
                           'performance': 0.8753977968176254},
                     '5': {'model': RAkELClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                      random_state=42,
                                                                                      solver='liblinear'),
                                                    n_estimators=12),
                           'name': 'RAKEL[LR] - ATT&CK-BERT',
                           'performance': 0.7576853526220615}})

In [8]:
with open("models/Computersecurity.pickle", "rb") as f:
    cs_mod = pickle.load(f)

cs_mod

ModuleSystem(mlb=MultiLabelBinarizer(),
             models={'0': {'model': LPClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'LP[RF] - DistilRoBERTa',
                           'performance': 0.9200913242009132},
                     '1': {'model': HOMERClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'HOMER[RF] - DistilRoBERTa',
                           'performance': 0.9247648902821317},
                     '2': {'model': BRClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                   random_state=42,
                                                                                   solver='liblinear')),
                           'name': 'BR[LR] - DistilRoBERTa',
                           'performance': 0.46153846153846156}})

In [9]:
with open("models/Operatingsystems.pickle", "rb") as f:
    os_mod = pickle.load(f)

os_mod

ModuleSystem(mlb=MultiLabelBinarizer(),
             models={'0': {'model': MBRClassifier(base_estimator=GaussianNB()),
                           'name': 'MBR[GNB] - ATT&CK-BERT',
                           'performance': 0.8943089430894309},
                     '1': {'model': CCClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                   random_state=42,
                                                                                   solver='liblinear')),
                           'name': 'CC[LR] - ATT&CK-BERT',
                           'performance': 0.8695652173913043},
                     '2': {'mod...
                           'performance': 0.9230769230769231},
                     '3': {'model': MBRClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                    random_state=42,
                                                                                    solver='liblinear')),
                           'name': 'MBR[LR] - ATT&CK-BERT',
                           'performance': 0.8},
                     '4': {'model': BRClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                   random_state=42,
                                                                                   solver='liblinear')),
                           'name': 'BR[LR] - ATT&CK-BERT',
                           'performance': 0.9210526315789473}})

In [10]:
with open("models/Software.pickle", "rb") as f:
    sw_mod = pickle.load(f)

sw_mod

ModuleSystem(mlb=MultiLabelBinarizer(),
             models={'0': {'model': LPClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'LP[RF] - DistilRoBERTa',
                           'performance': 0.8346774193548387},
                     '1': {'model': BRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'BR[RF] - DistilRoBERTa',
                           'performance': 0.9098901098901099}})

In [11]:
with open("models/Hardware.pickle", "rb") as f:
    hw_mod = pickle.load(f)

hw_mod

ModuleSystem(mlb=MultiLabelBinarizer(),
             models={'0': {'model': LPClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'LP[RF] - GloVe.6B.50D',
                           'performance': 0.9245585874799358},
                     '1': {'model': MBRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'MBR[RF] - GloVe.6B.50D',
                           'performance': 0.5581395348837209},
                     '2': {'model': BRClassifier(base_estimator=RandomForestClassifier(random_state=42)),
                           'name': 'BR[RF] - GloVe.6B.50D',
                           'performance': 0.6956521739130435},
                     '3': {'model': BRClassifier(base_estimator=LogisticRegression(max_iter=10000,
                                                                                   random_state=42,
                                                                                   solver='liblinear')),
                           'name': 'BR[LR] - GloVe.6B.50D',
                           'performance': 0.5555555555555556}})

In [41]:
class System(BaseEstimator, ClassifierMixin):
    def __init__(self, general, cs, os, sw, hw):
        self.general = general  # GloVe.6B.50D
        self.cs = cs  # DistilRoBERTa
        self.os = os  # ATT&CK-BERT
        self.sw = sw  # DistilRoBERTa
        self.hw = hw  # GloVe.6B.50D

    def predict(self, texts: pd.Series) -> ArrayLike:
        general_dim = len(self.general.mlb.classes_)
        cs_dim = len(self.cs.mlb.classes_) - 1
        os_dim = len(self.os.mlb.classes_) - 1
        sw_dim = len(self.sw.mlb.classes_) - 1
        hw_dim = len(self.hw.mlb.classes_) - 1
        Y_pred = np.zeros((texts.shape[0], general_dim + cs_dim + os_dim + sw_dim + hw_dim))
        X_DistilRoBERTa = preprocess_texts(
            list_str=texts,
            model_path='sentence-transformers/all-distilroberta-v1',
            embedding_dim=None,
        )
        X_ATTACK_BERT = preprocess_texts(
            list_str=texts,
            model_path='basel/ATTACK-BERT',
            embedding_dim=None,
        )
        X_GLOVE_6B_50D = preprocess_texts(
            list_str=texts,
            model_path='embedders/glove.6B.50d.txt',
            embedding_dim=50,
        )
        y_general = self.general.predict(X_ATTACK_BERT)
        Y_pred[:, :general_dim] = y_general
        for idx, t in enumerate(self.general.mlb.inverse_transform(y_general)):
            if 'other' in t:
                Y_pred[idx, :] = np.zeros((1, general_dim + cs_dim + os_dim + sw_dim + hw_dim))
                Y_pred[idx, 4:5] = 1.0
            if 'computer security' in t and 'other' not in t:
                y_cs = self.cs.predict(X_DistilRoBERTa[idx:idx + 1])
                Y_pred[idx, general_dim:general_dim + cs_dim] = y_cs[:, :cs_dim]
            if 'operating systems' in t and 'other' not in t:
                y_os = self.os.predict(X_ATTACK_BERT[idx:idx + 1])
                Y_pred[idx, general_dim + cs_dim:general_dim + cs_dim + os_dim] = np.hstack(
                    (y_os[:, :2], y_os[:, 3:]))
            if "software" in t and 'other' not in t:
                y_sw = self.sw.predict(X_DistilRoBERTa[idx:idx + 1])
                Y_pred[idx, general_dim + cs_dim + os_dim:general_dim + cs_dim + os_dim + sw_dim] = y_sw[:, :sw_dim]
            if "hardware" in t and 'other' not in t:
                y_hw = self.hw.predict(X_GLOVE_6B_50D[idx:idx + 1])
                Y_pred[idx, general_dim + cs_dim + os_dim + sw_dim:] = y_hw[:, :hw_dim]
            if len(t) == 0:
                Y_pred[idx, 4:5] = 1.0
        return Y_pred

    def predict_targets(self, texts: pd.Series) -> pd.Series:
        general_dim = len(self.general.mlb.classes_)
        cs_dim = len(self.cs.mlb.classes_) - 1
        os_dim = len(self.os.mlb.classes_) - 1
        sw_dim = len(self.sw.mlb.classes_) - 1
        Y_pred = self.predict(texts)
        y_general = Y_pred[:, :general_dim]
        y_cs = Y_pred[:, general_dim:general_dim + cs_dim]
        y_os = Y_pred[:, general_dim + cs_dim:general_dim + cs_dim + os_dim]
        y_sw = Y_pred[:, general_dim + cs_dim + os_dim:general_dim + cs_dim + os_dim + sw_dim]
        y_hw = Y_pred[:, general_dim + cs_dim + os_dim + sw_dim:]
        targets_general = self.general.mlb.inverse_transform(y_general)
        targets_cs = self.cs.mlb.inverse_transform(np.hstack((y_cs, np.zeros((y_cs.shape[0], 1)))))
        targets_os = self.os.mlb.inverse_transform(np.hstack((y_os[:, :2], np.zeros((y_os.shape[0], 1)), y_os[:, 2:])))
        targets_sw = self.sw.mlb.inverse_transform(np.hstack((y_sw, np.zeros((y_sw.shape[0], 1)))))
        targets_hw = self.hw.mlb.inverse_transform(np.hstack((y_hw, np.zeros((y_hw.shape[0], 1)))))
        return pd.Series(data=[list(set(target_general + target_cs + target_os + target_sw + target_hw)) for
                               target_general, target_cs, target_os, target_sw, target_hw in
                               zip(targets_general, targets_cs, targets_os, targets_sw, targets_hw)], index=texts.index)

    def evaluate(self, texts: pd.Series, targets: pd.Series) -> dict[str, float]:
        targets_general = self.general.mlb.transform(targets)
        targets_cs = self.cs.mlb.transform(targets)
        targets_os = self.os.mlb.transform(targets)
        targets_sw = self.sw.mlb.transform(targets)
        targets_hw = self.hw.mlb.transform(targets)
        Y_test = np.hstack((targets_general, targets_cs[:, :-1], targets_os[:, :2], targets_os[:, 3:],
                            targets_sw[:, :-1], targets_hw[:, :-1]))
        Y_pred = self.predict(texts)
        return assess(Y_test, Y_pred)

    def evaluate_filter(self, texts: pd.Series, targets: pd.Series) -> dict[str, float]:
        Y_test = pd.Series([1 if t else 0 for t in targets])
        Y_pred = self.predict(texts)
        return assess(Y_test, Y_pred[:, 4:5])

    def classes(self):
        return list(self.general.mlb.classes_) + list(self.cs.mlb.classes_[:-1]) + list(
            self.os.mlb.classes_[:2]) + list(self.os.mlb.classes_[3:]) + list(self.sw.mlb.classes_[:-1]) + list(
            self.hw.mlb.classes_[:-1])


solution = System(general_mod, cs_mod, os_mod, sw_mod, hw_mod)

In [42]:
pd.DataFrame(
    [solution.predict_targets(texts[:10]), targets[:10], texts[:10], test_tweets['relevant'][:10]],
    index=["Predicted", "Actual", "Text", "Relevant"]).transpose()

,Predicted,Actual,Text,Relevant
0,"[computer security, network security, databases, antivirus and malware, software]","[antivirus and malware, software, databases, computer security, network security]",threatmeter: irisgraphic sql injection vulnerability,True
1,[other],"[antivirus and malware, computer security]",Download this how-to guide on preparing for and detecting ransomware before it's too late:,False
2,"[antivirus and malware, computer security]","[antivirus and malware, computer security]",The US Justice Department is poised to charge North Korean hackers over the May 2017 global WannaCry ransomware att…,True
3,[other],[other],"3) Control and monitor compliance. Also, insurance policy exists within their inf…",False
4,"[databases, software]","[databases, software]","Ransomware down, but not out, report reveals",True
5,[other],[other],This is a brilliant article love that we are in all the right places …,False
6,[other],"[antivirus and malware, computer, hardware, computer security]","North Korean hackers sanctioned, facing charges for Sony hack, Wannacry ransomware attack",True
7,[other],[networking],"On the opposite end: Dumbledore is so larger-than-life, but we get to see his vulnerability (and crappy decisions)",True
8,[other],"[hardware, computer components]",I hope you run ransomware,True
9,[other],"[network security, computer security]",Did you catch our blog earlier this week about the WordPress vulnerability recently discovered by a researcher from…,False


In [45]:
system_performance = solution.evaluate(texts, targets)
print("==========================================================")
print("System Performance")
print("==========================================================")
print(f"Accuracy:\t{system_performance['accuracy'] * 100:.2f}%")
print(f"Hamming Loss:\t{system_performance['hamming_loss']:.3f}")
report = pd.DataFrame(system_performance["report"])
report.columns = solution.classes() + ["micro avg", "macro avg", "weighted avg", "samples avg"]
report = (report.transpose().map(lambda x: f"{x:.2f}" if isinstance(x, float) else x).to_string())
print(f"{report}")
print("==========================================================\n")

/home/cela/.local/programs/anaconda3/envs/msc-thesis/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['antivirus and malware', 'computer', 'computer components', 'computer networking', 'databases', 'linux', 'mac os', 'network security', 'portable computer', 'router', 'servers', 'unix', 'windows'] will be ignored
  warnings.warn(
/home/cela/.local/programs/anaconda3/envs/msc-thesis/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['computer', 'computer components', 'computer networking', 'computer security', 'databases', 'hardware', 'linux', 'mac os', 'networking', 'operating systems', 'portable computer', 'router', 'servers', 'software', 'unix', 'windows'] will be ignored
  warnings.warn(
/home/cela/.local/programs/anaconda3/envs/msc-thesis/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['antivirus and malware', 'computer', 'computer compone

System Performance
Accuracy:	59.04%
Hamming Loss:	0.079
                      precision recall f1-score  support   auc
computer security          0.94   0.42     0.58  1738.00  0.70
hardware                   0.97   0.38     0.54   461.00  0.69
networking                 0.79   0.33     0.47   240.00  0.66
operating systems          0.84   0.64     0.73   156.00  0.82
other                      0.55   0.98     0.71  1603.00  0.73
software                   0.88   0.54     0.67   950.00  0.76
antivirus and malware      0.92   0.51     0.66  1110.00  0.75
network security           0.91   0.26     0.41   817.00  0.63
linux                      0.88   0.73     0.79    77.00  0.86
mac os                     1.00   0.54     0.70    13.00  0.77
unix                       0.69   0.64     0.67    14.00  0.82
windows                    0.67   0.52     0.59    46.00  0.76
databases                  0.89   0.62     0.73   342.00  0.80
computer                   0.88   0.37     0.52   362.00  0.68

In [46]:
from sklearn.metrics import (
    classification_report,
    hamming_loss,
    roc_auc_score,
)

Y_test = pd.Series([1 if t else 0 for t in test_tweets['relevant']])
Y_pred = pd.Series([not t for t in solution.predict(texts)[:, 4:5]])
auc_score_macro = roc_auc_score(Y_test, Y_pred, average="macro")
auc_score_weighted = roc_auc_score(Y_test, Y_pred, average="weighted")

report = classification_report(Y_test, Y_pred, output_dict=True, zero_division=0.0)
report["macro avg"]["auc"] = auc_score_macro
report["weighted avg"]["auc"] = auc_score_weighted

hamming = hamming_loss(Y_test, Y_pred)
system_performance = {"hamming_loss": hamming, "report": report}

print("==========================================================")
print("System Performance")
print("==========================================================")
print(f"Hamming Loss:\t{system_performance['hamming_loss']:.3f}")
report = pd.DataFrame(system_performance["report"])
report.columns = ["not relevant", "relevant"] + ["accuracy", "macro avg", "weighted avg"]
report = (report.transpose().map(lambda x: f"{x:.2f}" if isinstance(x, float) else x).to_string())
print(f"{report}")
print("==========================================================\n")

System Performance
Hamming Loss:	0.274
             precision recall f1-score  support   auc
not relevant      0.66   0.93     0.77  2004.00   nan
relevant          0.89   0.53     0.66  2063.00   nan
accuracy          0.73   0.73     0.73     0.73  0.73
macro avg         0.77   0.73     0.72  4067.00  0.73
weighted avg      0.77   0.73     0.72  4067.00  0.73



In [34]:
pd.DataFrame(np.hstack((Y_test.values.reshape(-1, 1), Y_pred, test_tweets['text'][:end].values.reshape(-1, 1))), columns=["relevant", "predicted", "text"])

,relevant,predicted,text
0,0,0.0,threatmeter: irisgraphic sql injection vulnerability
1,1,1.0,Download this how-to guide on preparing for and detecting ransomware before it's too late:
2,0,0.0,The US Justice Department is poised to charge North Korean hackers over the May 2017 global WannaCry ransomware att…
3,1,1.0,"3) Control and monitor compliance. Also, insurance policy exists within their inf…"
4,0,0.0,"Ransomware down, but not out, report reveals"
...,...,...,...
995,1,0.0,"I can't wait 'till the DDOS detection things are put into place.....2 wins and the next match, little by little, te…"
996,0,0.0,Synacktiv publishes detailed report and PoC for an RCE flaw in one of the most popular WordPress plugins —Duplicato…
997,0,0.0,Vulnerability Found In Major Manufacturers Of Android Phones
998,1,1.0,In the corner of a mans mind there’s a corner where vulnerability hides


In [38]:
pd.DataFrame(data=solution.predict(pd.Series([
    "There are rootkits threats ongoing in the wild, and they're infecting the AMD CPUs when you open an EXE file attackers send via mail. Pay attention"])),
    columns=solution.classes())

,computer security,hardware,networking,operating systems,other,software,antivirus and malware,network security,linux,mac os,unix,windows,databases,computer,computer components,computer networking
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
